In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
# Ielādē apmācību un testa datus
df = pd.read_csv('dati_sez.csv', sep=';')
df_test = pd.read_csv('2024_sez.csv', sep=';')

In [3]:
# Izveido jaunu kolonu 'Telpu grupas vidējais stāvs'
df['Telpu grupas vidējais stāvs'] = (df['Telpu grupas zemākais stāvs'] + df['Telpu grupas augstākais stāvs']) / 2
df_test['Telpu grupas vidējais stāvs'] = (df_test['Telpu grupas zemākais stāvs'] + df_test['Telpu grupas augstākais stāvs']) / 2

In [ ]:
del_col = ['Darījuma summa, EUR', 'Būves apbūves laukums, m2', 'Būves būvtilpums, m3', 'Telpu grupas augstākais stāvs', 'Telpu grupas zemākais stāvs', 'Dzīvokļa kopplatība, m2']
# No apmācības un testa datu kopām atdala kolonnu 'Darījuma summa, EUR'
x_train = df.drop(columns=del_col) 
y_train = df['Darījuma summa, EUR'] 
x_test = df_test.drop(columns=del_col)
y_test_actual = df_test['Darījuma summa, EUR']

In [5]:
# Definē hiperparametru režģi GridSearchCV funkcijai
param_grid = {
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 3, 4],
    'min_samples_leaf': [1, 2, 5, 10],
}

# Izveido lēmumu koka regresijas modeli
dt_reg = DecisionTreeRegressor(random_state=42)

# Ar GridSearchCV veic labāko parametru meklēšanu
grid_search = GridSearchCV(dt_reg, param_grid, scoring='neg_root_mean_squared_error')  
grid_search.fit(x_train, y_train)  # Apmāca modeli ar apmācības datiem

# Iegūst un izdrukā labākos parametrus
best_params = grid_search.best_params_
print("Best Parameters:", best_params)


Best Parameters: {'max_depth': 5, 'min_samples_leaf': 5, 'min_samples_split': 2}


In [6]:
# Izveido lēmumu koka regresijas modeli, izmantojot labākos atrastos parametrus
best_dt_reg = DecisionTreeRegressor(**best_params, random_state=42)
# Apmāca šo modeli ar apmācību datiem
best_dt_reg.fit(x_train, y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=5, random_state=42)

In [7]:
# Izmanto apmācīto modeli, lai prognozētu cenas testa datiem
y_test_pred = best_dt_reg.predict(x_test)

In [8]:
# Novērtē modeļa precizitāti uz testa datiem
mae = mean_absolute_error(y_test_actual, y_test_pred)
mse = mean_squared_error(y_test_actual, y_test_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_actual, y_test_pred)

# Aprēķina relatīvās kļūdas
relative_errors = np.abs(y_test_pred - y_test_actual) / y_test_actual
mre = np.max(relative_errors)
are = np.mean(relative_errors)

In [9]:
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R^2: {r2}")
print(f"MRE: {mre}")
print(f"ARE: {are}")

MAE: 14208.775207133565
MSE: 357857746.8861972
RMSE: 18917.128399580026
R^2: 0.8465979116887443
MRE: 7.763766626506025
ARE: 0.523266104952672
